In [1]:
from model import Model
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler

## 1. Loading datasets

Checklist:
- Each matrix should be of type `double`
- Every `nan` should be removed
- `y` should be flattened

### 1.1. A9A

In [2]:
a9a_train_raw = pd.read_csv('datasets/a9a-train.tsv', sep=' ', header=None).dropna()
a9a_test_raw = pd.read_csv('datasets/a9a-test.tsv', sep=' ', header=None).dropna()

In [3]:
scaler = StandardScaler()

X_a9a_train = a9a_train_raw[a9a_train_raw.columns[1:-1]].values.astype(np.double)[:1000]
y_a9a_train = a9a_train_raw[a9a_train_raw.columns[0:1]].values.astype(np.double).flatten()[:1000]

X_a9a_train = scaler.fit_transform(X_a9a_train)

X_a9a_test = a9a_test_raw[a9a_test_raw.columns[1:-1]].values.astype(np.double)
y_a9a_test = a9a_test_raw[a9a_test_raw.columns[0:1]].values.astype(np.double).flatten()

X_a9a_test = scaler.transform(X_a9a_test)

In [4]:
X_a9a_train.shape

(1000, 13)

### 1.1. real-sim

## 2. Experiments

In [5]:
m = Model()

In [6]:
m.fit(X=X_a9a_train, y=y_a9a_train, optimizer='cvxopt')

Solve process failed.
Reason: "Terminated (maximum number of iterations reached)."


,iteration,pcost,dcost,gap,pres,dres
0,0,-4.898300e+02,-1.482600e+03,6.000000e+03,60.00000,3.0000
1,1,-1.736800e+03,-3.805000e+03,5.000000e+03,40.00000,2.0000
2,2,-5.435200e+03,-9.276000e+03,5.000000e+03,30.00000,2.0000
3,3,-1.455200e+04,-1.787600e+04,3.000000e+03,20.00000,1.0000
4,4,-8.057800e+04,-8.389800e+04,3.000000e+03,20.00000,1.0000
...,...,...,...,...,...,...
96,96,-2.283400e+12,-2.283400e+12,3.000000e-73,0.00030,0.0005
97,97,-2.283300e+12,-2.283300e+12,2.000000e-74,0.00010,0.0004
98,98,-2.283400e+12,-2.283400e+12,2.000000e-76,0.00030,0.0004
99,99,-2.283400e+12,-2.283400e+12,7.000000e-78,0.00004,0.0005


In [ ]:
y_hat = m.predict(X=X_a9a_test)

In [ ]:
y_hat = m.predict(X=X_a9a_test)
accuracy_score(y_hat, y_a9a_test)

In [ ]:
len(y_hat)

In [ ]:
np.sum(y_hat)

In [ ]:
m.w

In [43]:
m.w

array([[ 0.1446335 ],
       [-0.00059361],
       [ 0.01777087],
       [ 0.00663613],
       [ 0.2088539 ],
       [-0.27536726],
       [-0.01244485],
       [-0.12536744],
       [-0.05347449],
       [-0.03711447],
       [ 0.26659523],
       [-0.16539944],
       [ 0.12054069]])